In [191]:
import scanpy as sc
import pandas as pd
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.models.transforms import CustomJSTransform
from bokeh.transform import factor_cmap, transform
import sys
sys.path.insert(0, '../beehive/')
import numpy as np
from beehive import config, util, expset
from bokeh.palettes import Category20
import polars as pl

In [ ]:
output_notebook()

In [54]:
dataset_id = 'h.man2m.1.sct'

In [ ]:
expset.obslist(dataset_id)

In [56]:
def get_data(dataset_id, gene1, gene2, obsfield):
    rv = pd.DataFrame(dict(
        x = expset.get_gene(dataset_id, gene1)[:,0],
        y = expset.get_gene(dataset_id, gene2)[:,0],
        obs = expset.get_meta(dataset_id, obsfield)[:,0],
        ))
    return rv
    
data = get_data(dataset_id, 'TREM2', 'APOE', 'injection')
data

,x,y,obs
0,1.609438,3.295837,None
1,2.484907,2.833213,None
2,2.302585,3.806662,None
3,1.609438,2.890372,None
4,2.397895,3.465736,None
...,...,...,...
100826,0.000000,1.945910,oAb
100827,0.000000,0.693147,oAb
100828,1.098612,3.135494,oAb
100829,0.693147,2.639057,oAb


In [ ]:
p = figure()
ages = list(data['obs'].unique())
source = ColumnDataSource(data)
p.scatter(x='x', y='y', source=source,  legend_field="obs",
          fill_alpha=0.2, size=3,
         color=factor_cmap('obs', 'Category10_3', ages))
show(p)

In [ ]:
expset.get_varfields(dsid)

In [ ]:

datadir = util.get_datadir("h5ad")
dsid = "m.how1m.1"
#print(expset.get_varfields(dsid))
rv = pl.read_parquet(datadir / f"{dsid}.var.prq",["field"]+["APOE"])
rv.to_pandas()

In [45]:

datadir = util.get_datadir("h5ad")
dsid = "h.s423a.1"
varsfields = expset.get_varfields(dsid)
categ = "plaque_bin__all"
len(pl.read_parquet(datadir / f"{dsid}.var.prq").columns)
rv1 = pl.read_parquet(datadir / f"{dsid}.var.prq",["plaque_bin__all__lfc"])
rv2 = pl.read_parquet(datadir / f"{dsid}.var.prq",[12])
type(rv1)
type(rv2)
#rv
pl.concat([rv1,rv2],how="horizontal")

plaque_bin__all__lfc,__index_level_0__
f64,str
-0.001236,"""AL627309.1"""
0.119385,"""AL627309.5"""
0.011838,"""LINC01409"""
0.003292,"""LINC01128"""
0.0139,"""LINC00115"""
0.008326,"""FAM41C"""
0.0144,"""SAMD11"""
-0.003474,"""NOC2L"""
0.035005,"""KLHL17"""


In [ ]:
vars = expset.get_varfields(dsid)
unique_vars = list(set([x.replace("__lfc","").replace("__padj","") for x in vars]))

In [11]:
# test  = [(x,x) for x in unique_vars]



categ = "ptau_bin__all"
dsid = "h.s423a.1"
data = expset.get_dedata_new(dsid,categ)

categ_x  = categ + "__lfc"
categ_y = categ + "__padj"
data[categ_y] = np.log10(data[categ_y]) * (-1)
plot = figure()
plot.scatter(x = categ_x, y = categ_y, source = ColumnDataSource(data))
show(plot)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
len(expset.get_genes(dsid))

In [ ]:
expset.get_facet_options(dsid)

In [ ]:
# figure = sns.kdeplot(data=data, x="gene1", y="gene2",hue="obs", fill=False)


In [ ]:
data = pd.DataFrame(dict(
        gene1 = expset.get_gene(dataset_id, "APOE")[:,0],
        gene2 = expset.get_gene(dataset_id, "TREM2")[:,0],
        obs = expset.get_meta(dataset_id, "cell.state")[:,0],
        ))
data

In [ ]:
unique_obs = expset.get_obsfields(dataset_id)

In [ ]:
expset.get_dataset(dsid)

In [ ]:
hv.extension('bokeh')
def flatten(xss):
    return [x for xs in xss for x in xs]

unique_obs = pd.DataFrame(data)['obs'].unique()
index_cmap = factor_cmap('obs', Category20[len(unique_obs)], unique_obs)

images = []
for index,obs in enumerate(unique_obs):
    datasample = data.loc[(data.obs == obs)]
    img = hv.Bivariate(datasample)
    images = images + [img]
    fig = hv.render(img)

plot=figure()
plot.renderers = flatten([hv.render(x).renderers for x in images])

for index,x in enumerate(plot.renderers):
    x.glyph.line_color = index_cmap["transform"].palette[index]

show(plot)

In [ ]:
# hv.help(hv.Bivariate)
#fig.legend.items
#p.renderers = fig.renderers
hv.render(images[0]).renderers

In [ ]:
show(plot)

In [ ]:
# hv.Bivariate(datasample).opts(legend_labels={"test":"try"},)

merged_image = hv.NdOverlay({index: hv.Bivariate(data.loc[(data.obs == obs)], name=obs).opts(muted_fill_alpha=0)for index,obs in enumerate(unique_obs)})


In [ ]:
plot=figure(width = 1000,height = 1500)
plot = hv.render(merged_image)


In [ ]:
merged_image = hv.NdOverlay({index: hv.Bivariate(data.loc[(data.obs == obs)], name=obs).opts(muted_alpha=0,width = 1500, height = 900)for index,obs in enumerate(unique_obs)})
plot = hv.render(merged_image)

for index,obs in enumerate(unique_obs):
    plot.legend.items[index].label["value"] = obs
for index,rends in enumerate(plot.renderers):   
    rends.glyph.line_color = index_cmap["transform"].palette[index]

show(plot)

In [ ]:
for index,x in enumerate(plot.renderers):
    x.glyph.line_color = index_cmap["transform"].palette[index]


In [ ]:
plot.width = 600
plot.height = 600
show(plot)

In [52]:
datadir = util.get_datadir("h5ad")
dsid = "h.s423a.1"
#print(expset.get_varfields(dsid))
rv = pl.read_parquet(datadir / f"{dsid}.var.prq")
rv


plaque_bin__all__lfc,plaque_bin__all__padj,plaque_bin__alz__lfc,plaque_bin__alz__padj,plaque_bin__res__lfc,plaque_bin__res__padj,ptau_bin__all__lfc,ptau_bin__all__padj,ptau_bin__alz__lfc,ptau_bin__alz__padj,ptau_bin__res__lfc,ptau_bin__res__padj,__index_level_0__
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
-0.001236,0.000058,-0.020059,0.0,-0.020059,0.0,0.0137,0.000008,0.000983,0.000053,-0.027615,0.000012,"""AL627309.1"""
0.119385,5.6338e-15,0.085352,0.000001,0.085352,0.000001,0.080059,6.6723e-9,0.019779,0.000033,0.004381,0.000052,"""AL627309.5"""
0.011838,0.000035,-0.006136,0.000053,-0.006136,0.000053,0.0641,0.000001,-0.037134,0.000025,0.073206,0.000027,"""LINC01409"""
0.003292,0.000049,0.035708,0.000025,0.035708,0.000025,-0.056485,0.000005,-0.2751,1.0019e-12,0.009562,0.000054,"""LINC01128"""
0.0139,0.000002,0.0213,0.0,0.0213,0.0,0.015407,0.000003,0.001455,0.000048,-0.016627,0.000019,"""LINC00115"""
0.008326,0.000026,0.003975,0.000047,0.003975,0.000047,0.008096,0.000029,-0.000979,0.000055,-0.042663,0.000007,"""FAM41C"""
0.0144,0.000014,0.011848,0.000034,0.011848,0.000034,0.003749,0.000057,0.011754,0.0,-0.0499,0.000006,"""SAMD11"""
-0.003474,0.000054,0.132306,3.0003e-7,0.132306,3.0003e-7,-0.081222,6.4600e-7,-0.208302,4.5755e-10,0.292488,6.7999e-7,"""NOC2L"""
0.035005,0.000008,0.030311,0.000027,0.030311,0.000027,-0.0028,0.000052,-0.133004,1.6708e-7,0.011034,0.000055,"""KLHL17"""


In [192]:
p = figure()
data = pd.DataFrame(dict(
        gene1 = expset.get_gene(dataset_id, "APOE")[:,0],
        gene2 = expset.get_gene(dataset_id, "TREM2")[:,0],
        obs = expset.get_meta(dataset_id, "cell.state")[:,0],
        ))

bins = hexbin(data["gene1"], data["gene2"], 0.1)
source_bin = ColumnDataSource(data=dict(r=[], q=[], counts=[]))
source_bin.data = dict(r=bins.r, q=bins.q, counts=bins.counts)

v_func_alpha  = """
var new_xs = new Array(xs.length)
for(var i = 0; i < xs.length; i++) {
    new_xs[i] = alpha_map[xs[i]]
}
return new_xs
"""
length = len(sorted(source_bin.data["counts"].unique().tolist()))
alpha_map = dict(zip(sorted(source_bin.data["counts"].unique().tolist()),[x/length for x in range(0,length)]))
numerical_alpha_transformer = CustomJSTransform(args={"alpha_map": alpha_map}, v_func=v_func_alpha)


#plot:
t = p.hex_tile(q="q",r="r",size=0.1,source=bins,
alpha = transform("counts",numerical_alpha_transformer),line_color = None, color = "black")
# p.scatter(x = "gene1",y="gene2", source = ColumnDataSource(data),size = 2)
show(p)
